Σε αυτό το notebook κάνω ανάλυση των συνόλων δεδομένων CIFAR-100 με την χρήση της μεθόδου KPCA+LDA. Λειτουργώ με παρόμοιο τρόπο όπως και στην ανάλυση του CIFAR-10. Για περισσότερες λεπτομέριες, μπορείτε να ανατρέξετε στο αρχείο KPCA+LDA_CIFAR-10.ipynb. 

# CIFAR-100 analysis using KPCA+LDA
## Dataset import and preprocess
Χρησιμοποιώ τις οδηγίες στην ιστοσελίδα https://www.cs.toronto.edu/~kriz/cifar.html για την εγκατάσταση και φόρτωμα των δεδομένων του CIFAR-10 και CIFAR-100. Κάνω ένα απλό απλό min-max scaling των τιμών των δεδομένων στο $[0, 1]$.

In [18]:
import pickle

def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def fetch_cifar100_data(filename):
    decoded_data = unpickle(filename)
    
    # fine labels -> version with the higher amount of classification classes
    return decoded_data[bytes('data', 'utf-8')], decoded_data[bytes('fine_labels', 'utf-8')]

In [19]:
batch_filenames = [ 'cifar100/data_batch_1', 'cifar10/data_batch_2', 'cifar10/data_batch_3',
                    'cifar10/data_batch_4', 'cifar10/data_batch_5']

train_data, train_labels = fetch_cifar100_data('cifar100/train')
test_data, test_labels = fetch_cifar100_data('cifar100/test')

In [20]:
import numpy as np

In [36]:
X, y = np.array(train_data), np.array(train_labels)
X_test, y_test = np.array(test_data), np.array(test_labels)

Λόγω μεγέθους το dataset, χρησιμοποιώ 5.000 δεδομένα για training και άλλα 5.000 για validation.

In [37]:
from sklearn.model_selection import train_test_split

used_dataset_size = 5000 # amount of samples to retain

X, _, y, _ = train_test_split(X, y, train_size=used_dataset_size) 

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=.5) # 50-50 split

In [38]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(2500, 3072) (2500,) (2500, 3072) (2500,) (10000, 3072) (10000,)


Rescale:

In [39]:
X_train, X_val, X_test = X_train / 255, X_val / 255, X_test / 255

## Analysis
Ξεικάω με την εφαρμογή ΚΝΝ (με $k=5$) και NearestCentroid στον αρχικό χώρο και σε απλό PCA με διατήρηση 90\% της πληροφορίας.

In [40]:
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.metrics import accuracy_score

In [41]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print(f'KNN: {accuracy_score(y_pred, y_test)}')

nc = NearestCentroid()
nc.fit(X_train, y_train)

y_pred = nc.predict(X_test)
print(f'NC: {accuracy_score(y_pred, y_test)}')

KNN: 0.0715
NC: 0.0879


### PCA

In [43]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.9)

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_pca, y_train)

y_pred = knn.predict(X_test_pca)
print(f'KNN: {accuracy_score(y_pred, y_test)}')

nc = NearestCentroid()
nc.fit(X_train_pca, y_train)

y_pred = nc.predict(X_test_pca)
print(f'NC: {accuracy_score(y_pred, y_test)}')

KNN: 0.0771
NC: 0.0868


In [44]:
from sklearn.decomposition import KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def kpca_plus_lda(kpca_model, lda_model_in=None):
    X_train_kpca = kpca_model.fit_transform(X_train)
    X_val_kpca = kpca_model.transform(X_val)
    
    lda_model = lda_model_in if lda_model_in is not None else LinearDiscriminantAnalysis()
    
    X_train_lda = lda_model.fit_transform(X_train_kpca, y_train)
    X_val_lda = lda_model.transform(X_val_kpca)
    
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train_lda, y_train)

    y_pred = knn.predict(X_val_lda)
    knn_acc = accuracy_score(y_pred, y_val)

    nc = NearestCentroid()
    nc.fit(X_train_lda, y_train)

    y_pred = nc.predict(X_val_lda)
    nc_acc = accuracy_score(y_pred, y_val)
    
    return knn_acc, nc_acc

### PCA + LDA
Ξεκινάω με δοκιμή της απλής περίπτωσης PCA + LDA

In [45]:
pca_model = PCA(n_components=.9)

print('PCA + LDA:')

lda_model = LinearDiscriminantAnalysis()

X_train_lda = lda_model.fit_transform(X_train_pca, y_train)
X_test_lda = lda_model.transform(X_test_pca)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_lda, y_train)

y_pred = knn.predict(X_test_lda)
print(f'KNN: {accuracy_score(y_pred, y_test)}')

nc = NearestCentroid()
nc.fit(X_train_lda, y_train)

y_pred = nc.predict(X_test_lda)
print(f'NC: {accuracy_score(y_pred, y_test)}')

PCA + LDA:
KNN: 0.0658
NC: 0.1106


Παρατηρούμε ήδη πολύ καλύ βελτίωση σε σχέση με την απλή PCA.

### RBF Kernel
Δοκιμάζω αρικά με μεγάλες τιμές:

In [46]:
for gamma in [ 1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5 ]:
    for n_components in [ 500, 1000, 1500, 2000, 2500 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='rbf', gamma=gamma)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)
        
        print(f'RBF with n_components={n_components}, gamma={gamma}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')


RBF with n_components=500, gamma=1: KNN: 1.20%, NC: 0.92%
RBF with n_components=1000, gamma=1: KNN: 1.20%, NC: 0.92%
RBF with n_components=1500, gamma=1: KNN: 1.12%, NC: 1.28%
RBF with n_components=2000, gamma=1: KNN: 1.20%, NC: 1.20%
RBF with n_components=2500, gamma=1: KNN: 1.00%, NC: 1.24%
RBF with n_components=500, gamma=0.1: KNN: 1.28%, NC: 0.92%
RBF with n_components=1000, gamma=0.1: KNN: 1.24%, NC: 0.92%
RBF with n_components=1500, gamma=0.1: KNN: 0.76%, NC: 1.28%
RBF with n_components=2000, gamma=0.1: KNN: 1.12%, NC: 1.20%
RBF with n_components=2500, gamma=0.1: KNN: 0.72%, NC: 2.24%
RBF with n_components=500, gamma=0.01: KNN: 7.12%, NC: 11.36%
RBF with n_components=1000, gamma=0.01: KNN: 7.20%, NC: 9.84%
RBF with n_components=1500, gamma=0.01: KNN: 7.32%, NC: 8.60%
RBF with n_components=2000, gamma=0.01: KNN: 5.80%, NC: 6.04%
RBF with n_components=2500, gamma=0.01: KNN: 5.00%, NC: 10.20%
RBF with n_components=500, gamma=0.001: KNN: 6.52%, NC: 12.00%
RBF with n_components=1000, 

Πολύ καλή απόδοση παίρνουμε για `n_components=1000, gamma=0.001: KNN: 7.20%, NC: 11.20%`. Δοκιμάζω ένα ακόμη grid search.

In [54]:
for gamma in [ 0.0008, 0.001, 0.0012 ]:
    for n_components in [ 900, 1000, 1100 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='rbf', gamma=gamma)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'RBF with n_components={n_components}, gamma={gamma}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

RBF with n_components=900, gamma=0.0008: KNN: 7.20%, NC: 11.24%
RBF with n_components=1000, gamma=0.0008: KNN: 7.24%, NC: 11.20%
RBF with n_components=1100, gamma=0.0008: KNN: 7.52%, NC: 10.92%
RBF with n_components=900, gamma=0.001: KNN: 7.28%, NC: 11.40%
RBF with n_components=1000, gamma=0.001: KNN: 7.20%, NC: 11.20%
RBF with n_components=1100, gamma=0.001: KNN: 7.60%, NC: 10.92%
RBF with n_components=900, gamma=0.0012: KNN: 7.84%, NC: 11.56%
RBF with n_components=1000, gamma=0.0012: KNN: 7.32%, NC: 11.56%
RBF with n_components=1100, gamma=0.0012: KNN: 7.60%, NC: 11.40%


`n_components=900, gamma=0.0012: KNN: 7.84%, NC: 11.56%`
### Poly deg 2
Συνεχίζω με ανάλυση για πολυωνυμικά κerel 2ου βαθμού.

In [47]:
for coef0 in [ 100, 10, 1, 1e-1, 1e-2, 1e-3 ]:
    for n_components in [ 500, 1000, 1500, 2000, 2500 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=2, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 2 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 2 with n_components=500, coef0=100: KNN: 4.24%, NC: 8.12%
Poly deg 2 with n_components=1000, coef0=100: KNN: 4.16%, NC: 6.72%
Poly deg 2 with n_components=1500, coef0=100: KNN: 4.48%, NC: 5.80%
Poly deg 2 with n_components=2000, coef0=100: KNN: 4.64%, NC: 5.28%
Poly deg 2 with n_components=2500, coef0=100: KNN: 1.32%, NC: 4.00%
Poly deg 2 with n_components=500, coef0=10: KNN: 4.84%, NC: 8.80%
Poly deg 2 with n_components=1000, coef0=10: KNN: 4.68%, NC: 7.84%
Poly deg 2 with n_components=1500, coef0=10: KNN: 5.64%, NC: 7.36%
Poly deg 2 with n_components=2000, coef0=10: KNN: 5.60%, NC: 6.40%
Poly deg 2 with n_components=2500, coef0=10: KNN: 1.28%, NC: 5.20%
Poly deg 2 with n_components=500, coef0=1: KNN: 5.44%, NC: 10.08%
Poly deg 2 with n_components=1000, coef0=1: KNN: 5.52%, NC: 9.12%
Poly deg 2 with n_components=1500, coef0=1: KNN: 5.92%, NC: 8.20%
Poly deg 2 with n_components=2000, coef0=1: KNN: 6.36%, NC: 7.28%
Poly deg 2 with n_components=2500, coef0=1: KNN: 1.72%, NC: 5.4

* `n_components=1500, coef0=0.01: KNN: 7.12%, NC: 8.48%`
* `n_components=500, coef0=0.01: KNN: 6.20%, NC: 10.92%`

In [53]:
for coef0 in [ .0005, .001, .004, .007, .01, .013 ]:
    for n_components in [ 400, 500, 700, 900, 1100, 1300, 1500, 1700 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=2, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 2 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 2 with n_components=400, coef0=0.0005: KNN: 5.32%, NC: 10.68%
Poly deg 2 with n_components=500, coef0=0.0005: KNN: 5.80%, NC: 10.20%
Poly deg 2 with n_components=700, coef0=0.0005: KNN: 5.80%, NC: 9.80%
Poly deg 2 with n_components=900, coef0=0.0005: KNN: 5.64%, NC: 9.88%
Poly deg 2 with n_components=1100, coef0=0.0005: KNN: 5.96%, NC: 9.36%
Poly deg 2 with n_components=1300, coef0=0.0005: KNN: 6.20%, NC: 8.92%
Poly deg 2 with n_components=1500, coef0=0.0005: KNN: 7.04%, NC: 8.00%
Poly deg 2 with n_components=1700, coef0=0.0005: KNN: 6.20%, NC: 7.88%
Poly deg 2 with n_components=400, coef0=0.001: KNN: 5.24%, NC: 10.76%
Poly deg 2 with n_components=500, coef0=0.001: KNN: 5.80%, NC: 10.12%
Poly deg 2 with n_components=700, coef0=0.001: KNN: 5.76%, NC: 9.88%
Poly deg 2 with n_components=900, coef0=0.001: KNN: 5.64%, NC: 9.88%
Poly deg 2 with n_components=1100, coef0=0.001: KNN: 5.88%, NC: 9.52%
Poly deg 2 with n_components=1300, coef0=0.001: KNN: 6.52%, NC: 9.04%
Poly deg 2 with 

`n_components=400, coef0=0.007: KNN: 5.44%, NC: 11.00%`

In [60]:
for coef0 in [ .012, .013, .014, .015 ]:
    for n_components in [ 1200, 1300, 1400 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=2, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 2 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 2 with n_components=1200, coef0=0.012: KNN: 6.88%, NC: 10.08%
Poly deg 2 with n_components=1300, coef0=0.012: KNN: 7.48%, NC: 9.40%
Poly deg 2 with n_components=1400, coef0=0.012: KNN: 7.48%, NC: 8.88%
Poly deg 2 with n_components=1200, coef0=0.013: KNN: 7.00%, NC: 10.08%
Poly deg 2 with n_components=1300, coef0=0.013: KNN: 7.36%, NC: 9.48%
Poly deg 2 with n_components=1400, coef0=0.013: KNN: 7.40%, NC: 8.92%
Poly deg 2 with n_components=1200, coef0=0.014: KNN: 7.00%, NC: 10.04%
Poly deg 2 with n_components=1300, coef0=0.014: KNN: 7.48%, NC: 9.40%
Poly deg 2 with n_components=1400, coef0=0.014: KNN: 7.24%, NC: 8.88%
Poly deg 2 with n_components=1200, coef0=0.015: KNN: 7.08%, NC: 10.12%
Poly deg 2 with n_components=1300, coef0=0.015: KNN: 7.44%, NC: 9.44%
Poly deg 2 with n_components=1400, coef0=0.015: KNN: 7.24%, NC: 8.96%


### Poly deg 3
Δοκιμάζω το ίδιο με πολυώνυμα 3ου βαθμού.

In [48]:
for coef0 in [ 100, 10, 1, 1e-1, 1e-2, 1e-3 ]:
    for n_components in [ 500, 1000, 1500, 2000, 2500 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=3, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 3 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 3 with n_components=500, coef0=100: KNN: 4.12%, NC: 8.16%
Poly deg 3 with n_components=1000, coef0=100: KNN: 4.48%, NC: 6.88%
Poly deg 3 with n_components=1500, coef0=100: KNN: 4.52%, NC: 6.12%
Poly deg 3 with n_components=2000, coef0=100: KNN: 5.16%, NC: 5.36%
Poly deg 3 with n_components=2500, coef0=100: KNN: 1.88%, NC: 4.92%
Poly deg 3 with n_components=500, coef0=10: KNN: 4.96%, NC: 9.32%
Poly deg 3 with n_components=1000, coef0=10: KNN: 5.16%, NC: 8.36%
Poly deg 3 with n_components=1500, coef0=10: KNN: 5.84%, NC: 7.60%
Poly deg 3 with n_components=2000, coef0=10: KNN: 5.96%, NC: 6.72%
Poly deg 3 with n_components=2500, coef0=10: KNN: 1.76%, NC: 5.16%
Poly deg 3 with n_components=500, coef0=1: KNN: 5.96%, NC: 10.68%
Poly deg 3 with n_components=1000, coef0=1: KNN: 6.28%, NC: 9.64%
Poly deg 3 with n_components=1500, coef0=1: KNN: 6.52%, NC: 9.20%
Poly deg 3 with n_components=2000, coef0=1: KNN: 5.84%, NC: 7.88%
Poly deg 3 with n_components=2500, coef0=1: KNN: 1.60%, NC: 6.0

* `n_components=1000, coef0=0.1: KNN: 7.40%, NC: 9.56%`
* `n_components=500, coef0=0.1: KNN: 6.72%, NC: 10.88%`

In [55]:
for coef0 in [ .08, .09, .1, .11, .12 ]:
    for n_components in [ 800, 900, 1000, 1100, 1200 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=3, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 3 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 3 with n_components=800, coef0=0.08: KNN: 7.20%, NC: 10.08%
Poly deg 3 with n_components=900, coef0=0.08: KNN: 7.12%, NC: 9.68%
Poly deg 3 with n_components=1000, coef0=0.08: KNN: 7.44%, NC: 9.64%
Poly deg 3 with n_components=1100, coef0=0.08: KNN: 7.88%, NC: 9.56%
Poly deg 3 with n_components=1200, coef0=0.08: KNN: 7.84%, NC: 9.64%
Poly deg 3 with n_components=800, coef0=0.09: KNN: 7.28%, NC: 10.16%
Poly deg 3 with n_components=900, coef0=0.09: KNN: 7.20%, NC: 9.88%
Poly deg 3 with n_components=1000, coef0=0.09: KNN: 7.36%, NC: 9.56%
Poly deg 3 with n_components=1100, coef0=0.09: KNN: 7.92%, NC: 9.40%
Poly deg 3 with n_components=1200, coef0=0.09: KNN: 7.68%, NC: 9.64%
Poly deg 3 with n_components=800, coef0=0.1: KNN: 6.96%, NC: 10.20%
Poly deg 3 with n_components=900, coef0=0.1: KNN: 7.16%, NC: 9.84%
Poly deg 3 with n_components=1000, coef0=0.1: KNN: 7.40%, NC: 9.56%
Poly deg 3 with n_components=1100, coef0=0.1: KNN: 7.76%, NC: 9.48%
Poly deg 3 with n_components=1200, coef0=

`n_components=1200, coef0=0.08: KNN: 7.84%, NC: 9.64%`

In [62]:
for coef0 in [ .08, .09, .1, .11, .12 ]:
    for n_components in [ 300, 400, 500, 600, 700 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=3, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 3 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 3 with n_components=300, coef0=0.08: KNN: 6.44%, NC: 11.32%
Poly deg 3 with n_components=400, coef0=0.08: KNN: 7.08%, NC: 10.92%
Poly deg 3 with n_components=500, coef0=0.08: KNN: 6.68%, NC: 10.88%
Poly deg 3 with n_components=600, coef0=0.08: KNN: 7.04%, NC: 10.36%
Poly deg 3 with n_components=700, coef0=0.08: KNN: 6.84%, NC: 10.16%
Poly deg 3 with n_components=300, coef0=0.09: KNN: 6.72%, NC: 11.20%
Poly deg 3 with n_components=400, coef0=0.09: KNN: 7.24%, NC: 11.00%
Poly deg 3 with n_components=500, coef0=0.09: KNN: 6.72%, NC: 10.96%
Poly deg 3 with n_components=600, coef0=0.09: KNN: 6.88%, NC: 10.44%
Poly deg 3 with n_components=700, coef0=0.09: KNN: 6.56%, NC: 10.28%
Poly deg 3 with n_components=300, coef0=0.1: KNN: 6.80%, NC: 11.20%
Poly deg 3 with n_components=400, coef0=0.1: KNN: 7.04%, NC: 11.00%
Poly deg 3 with n_components=500, coef0=0.1: KNN: 6.72%, NC: 10.88%
Poly deg 3 with n_components=600, coef0=0.1: KNN: 6.88%, NC: 10.52%
Poly deg 3 with n_components=700, coef

`n_components=400, coef0=0.09: KNN: 7.24%, NC: 11.00%`

### Poly deg 5
Συνεχίζω με πολυωνυμικά kernel βαθμού 5.

In [49]:
for coef0 in [ 100, 10, 1, 1e-1, 1e-2, 1e-3 ]:
    for n_components in [ 500, 1000, 1500, 2000, 2500 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=5, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 5 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 5 with n_components=500, coef0=100: KNN: 4.28%, NC: 8.36%
Poly deg 5 with n_components=1000, coef0=100: KNN: 4.52%, NC: 7.36%
Poly deg 5 with n_components=1500, coef0=100: KNN: 4.92%, NC: 6.60%
Poly deg 5 with n_components=2000, coef0=100: KNN: 5.20%, NC: 5.32%
Poly deg 5 with n_components=2500, coef0=100: KNN: 1.20%, NC: 4.68%
Poly deg 5 with n_components=500, coef0=10: KNN: 5.52%, NC: 10.04%
Poly deg 5 with n_components=1000, coef0=10: KNN: 5.72%, NC: 8.56%
Poly deg 5 with n_components=1500, coef0=10: KNN: 6.00%, NC: 7.88%
Poly deg 5 with n_components=2000, coef0=10: KNN: 6.08%, NC: 6.76%
Poly deg 5 with n_components=2500, coef0=10: KNN: 1.88%, NC: 5.68%
Poly deg 5 with n_components=500, coef0=1: KNN: 6.12%, NC: 11.20%
Poly deg 5 with n_components=1000, coef0=1: KNN: 6.88%, NC: 9.16%
Poly deg 5 with n_components=1500, coef0=1: KNN: 7.28%, NC: 8.76%
Poly deg 5 with n_components=2000, coef0=1: KNN: 6.48%, NC: 7.84%
Poly deg 5 with n_components=2500, coef0=1: KNN: 2.76%, NC: 5.

* `n_components=500, coef0=0.1: KNN: 7.92%, NC: 9.04%`
* `n_components=500, coef0=1: KNN: 6.12%, NC: 11.20%`

In [56]:
for coef0 in [ .08, .09, .1, .11, .12 ]:
    for n_components in [ 300, 400, 500, 600, 700 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=5, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 5 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 5 with n_components=300, coef0=0.08: KNN: 6.76%, NC: 9.44%
Poly deg 5 with n_components=400, coef0=0.08: KNN: 7.40%, NC: 9.16%
Poly deg 5 with n_components=500, coef0=0.08: KNN: 7.52%, NC: 8.76%
Poly deg 5 with n_components=600, coef0=0.08: KNN: 7.96%, NC: 8.56%
Poly deg 5 with n_components=700, coef0=0.08: KNN: 8.48%, NC: 8.48%
Poly deg 5 with n_components=300, coef0=0.09: KNN: 6.88%, NC: 9.52%
Poly deg 5 with n_components=400, coef0=0.09: KNN: 7.24%, NC: 9.20%
Poly deg 5 with n_components=500, coef0=0.09: KNN: 7.56%, NC: 8.84%
Poly deg 5 with n_components=600, coef0=0.09: KNN: 7.80%, NC: 8.76%
Poly deg 5 with n_components=700, coef0=0.09: KNN: 8.48%, NC: 8.52%
Poly deg 5 with n_components=300, coef0=0.1: KNN: 7.12%, NC: 9.28%
Poly deg 5 with n_components=400, coef0=0.1: KNN: 7.32%, NC: 9.52%
Poly deg 5 with n_components=500, coef0=0.1: KNN: 7.92%, NC: 9.04%
Poly deg 5 with n_components=600, coef0=0.1: KNN: 7.84%, NC: 8.80%
Poly deg 5 with n_components=700, coef0=0.1: KNN: 8.

In [63]:
for coef0 in [ .8, .9, 1, 1.1, 1.2 ]:
    for n_components in [ 300, 400, 500, 600, 700 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=5, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 5 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 5 with n_components=300, coef0=0.8: KNN: 5.48%, NC: 11.96%
Poly deg 5 with n_components=400, coef0=0.8: KNN: 6.24%, NC: 11.08%
Poly deg 5 with n_components=500, coef0=0.8: KNN: 5.92%, NC: 11.20%
Poly deg 5 with n_components=600, coef0=0.8: KNN: 6.56%, NC: 11.08%
Poly deg 5 with n_components=700, coef0=0.8: KNN: 6.44%, NC: 10.12%
Poly deg 5 with n_components=300, coef0=0.9: KNN: 5.44%, NC: 12.16%
Poly deg 5 with n_components=400, coef0=0.9: KNN: 6.16%, NC: 10.92%
Poly deg 5 with n_components=500, coef0=0.9: KNN: 6.00%, NC: 11.20%
Poly deg 5 with n_components=600, coef0=0.9: KNN: 6.60%, NC: 10.80%
Poly deg 5 with n_components=700, coef0=0.9: KNN: 6.48%, NC: 10.16%
Poly deg 5 with n_components=300, coef0=1: KNN: 5.32%, NC: 11.72%
Poly deg 5 with n_components=400, coef0=1: KNN: 5.84%, NC: 10.68%
Poly deg 5 with n_components=500, coef0=1: KNN: 6.12%, NC: 11.20%
Poly deg 5 with n_components=600, coef0=1: KNN: 6.52%, NC: 10.92%
Poly deg 5 with n_components=700, coef0=1: KNN: 6.60%, N

`n_components=300, coef0=0.9: KNN: 5.44%, NC: 12.16%`

### Poly deg 7

In [50]:
for coef0 in [ 100, 10, 1, 1e-1, 1e-2, 1e-3 ]:
    for n_components in [ 500, 1000, 1500, 2000, 2500 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=7, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 7 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 7 with n_components=500, coef0=100: KNN: 4.16%, NC: 8.60%
Poly deg 7 with n_components=1000, coef0=100: KNN: 4.56%, NC: 7.24%
Poly deg 7 with n_components=1500, coef0=100: KNN: 5.12%, NC: 6.64%
Poly deg 7 with n_components=2000, coef0=100: KNN: 5.32%, NC: 5.72%
Poly deg 7 with n_components=2500, coef0=100: KNN: 1.56%, NC: 5.12%
Poly deg 7 with n_components=500, coef0=10: KNN: 5.56%, NC: 9.88%
Poly deg 7 with n_components=1000, coef0=10: KNN: 5.96%, NC: 9.04%
Poly deg 7 with n_components=1500, coef0=10: KNN: 5.72%, NC: 8.36%
Poly deg 7 with n_components=2000, coef0=10: KNN: 6.28%, NC: 7.20%
Poly deg 7 with n_components=2500, coef0=10: KNN: 1.80%, NC: 5.52%
Poly deg 7 with n_components=500, coef0=1: KNN: 6.24%, NC: 11.00%
Poly deg 7 with n_components=1000, coef0=1: KNN: 6.96%, NC: 9.72%
Poly deg 7 with n_components=1500, coef0=1: KNN: 7.36%, NC: 8.32%
Poly deg 7 with n_components=2000, coef0=1: KNN: 6.32%, NC: 7.56%
Poly deg 7 with n_components=2500, coef0=1: KNN: 1.88%, NC: 5.9

* `n_components=500, coef0=1: KNN: 6.24%, NC: 11.00%`
* `n_components=1500, coef0=1: KNN: 7.36%, NC: 8.32%`

In [57]:
for coef0 in [ .8, .9, 1, 1.1, 1.2  ]:
    for n_components in [ 400, 500, 700, 900, 1100, 1300, 1500, 1700 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=7, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 7 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 7 with n_components=400, coef0=0.8: KNN: 7.04%, NC: 11.04%
Poly deg 7 with n_components=500, coef0=0.8: KNN: 6.76%, NC: 11.20%
Poly deg 7 with n_components=700, coef0=0.8: KNN: 7.40%, NC: 10.12%
Poly deg 7 with n_components=900, coef0=0.8: KNN: 7.44%, NC: 9.44%
Poly deg 7 with n_components=1100, coef0=0.8: KNN: 7.32%, NC: 9.36%
Poly deg 7 with n_components=1300, coef0=0.8: KNN: 7.88%, NC: 8.92%
Poly deg 7 with n_components=1500, coef0=0.8: KNN: 7.12%, NC: 8.44%
Poly deg 7 with n_components=1700, coef0=0.8: KNN: 7.64%, NC: 8.32%
Poly deg 7 with n_components=400, coef0=0.9: KNN: 6.64%, NC: 10.84%
Poly deg 7 with n_components=500, coef0=0.9: KNN: 6.52%, NC: 11.08%
Poly deg 7 with n_components=700, coef0=0.9: KNN: 7.24%, NC: 10.16%
Poly deg 7 with n_components=900, coef0=0.9: KNN: 7.00%, NC: 9.36%
Poly deg 7 with n_components=1100, coef0=0.9: KNN: 7.20%, NC: 9.48%
Poly deg 7 with n_components=1300, coef0=0.9: KNN: 7.52%, NC: 9.24%
Poly deg 7 with n_components=1500, coef0=0.9: KNN:

`n_components=400, coef0=1: KNN: 7.04%, NC: 11.00%`

### Poly deg 9

In [51]:
for coef0 in [ 100, 10, 1, 1e-1, 1e-2, 1e-3 ]:
    for n_components in [ 500, 1000, 1500, 2000, 2500 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=9, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 9 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 9 with n_components=500, coef0=100: KNN: 4.52%, NC: 8.72%
Poly deg 9 with n_components=1000, coef0=100: KNN: 4.96%, NC: 7.72%
Poly deg 9 with n_components=1500, coef0=100: KNN: 5.60%, NC: 7.16%
Poly deg 9 with n_components=2000, coef0=100: KNN: 5.52%, NC: 6.20%
Poly deg 9 with n_components=2500, coef0=100: KNN: 1.80%, NC: 5.08%
Poly deg 9 with n_components=500, coef0=10: KNN: 5.72%, NC: 10.16%
Poly deg 9 with n_components=1000, coef0=10: KNN: 6.00%, NC: 9.16%
Poly deg 9 with n_components=1500, coef0=10: KNN: 6.00%, NC: 8.56%
Poly deg 9 with n_components=2000, coef0=10: KNN: 6.32%, NC: 7.56%
Poly deg 9 with n_components=2500, coef0=10: KNN: 1.72%, NC: 5.28%
Poly deg 9 with n_components=500, coef0=1: KNN: 7.20%, NC: 10.64%
Poly deg 9 with n_components=1000, coef0=1: KNN: 7.08%, NC: 9.32%
Poly deg 9 with n_components=1500, coef0=1: KNN: 7.36%, NC: 8.60%
Poly deg 9 with n_components=2000, coef0=1: KNN: 6.56%, NC: 7.68%
Poly deg 9 with n_components=2500, coef0=1: KNN: 2.28%, NC: 5.

`n_components=500, coef0=1: KNN: 7.20%, NC: 10.64%`

In [58]:
for coef0 in [ .8, .9, 1, 1.1, 1.2 ]:
    for n_components in [ 300, 400, 500, 600, 700 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=9, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 9 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 9 with n_components=300, coef0=0.8: KNN: 7.12%, NC: 10.56%
Poly deg 9 with n_components=400, coef0=0.8: KNN: 7.52%, NC: 10.20%
Poly deg 9 with n_components=500, coef0=0.8: KNN: 7.12%, NC: 9.84%
Poly deg 9 with n_components=600, coef0=0.8: KNN: 7.36%, NC: 9.60%
Poly deg 9 with n_components=700, coef0=0.8: KNN: 7.84%, NC: 9.76%
Poly deg 9 with n_components=300, coef0=0.9: KNN: 7.32%, NC: 10.52%
Poly deg 9 with n_components=400, coef0=0.9: KNN: 7.44%, NC: 10.40%
Poly deg 9 with n_components=500, coef0=0.9: KNN: 7.08%, NC: 9.96%
Poly deg 9 with n_components=600, coef0=0.9: KNN: 7.16%, NC: 10.04%
Poly deg 9 with n_components=700, coef0=0.9: KNN: 7.44%, NC: 9.80%
Poly deg 9 with n_components=300, coef0=1: KNN: 7.08%, NC: 10.84%
Poly deg 9 with n_components=400, coef0=1: KNN: 7.48%, NC: 10.44%
Poly deg 9 with n_components=500, coef0=1: KNN: 7.20%, NC: 10.64%
Poly deg 9 with n_components=600, coef0=1: KNN: 7.08%, NC: 10.16%
Poly deg 9 with n_components=700, coef0=1: KNN: 7.60%, NC: 9.

Δεν παρατηρούμε μεγάλες διαφορές με το προηγούμενο.

### Poly deg 11

In [52]:
for coef0 in [ 100, 10, 1, 1e-1, 1e-2, 1e-3 ]:
    for n_components in [ 500, 1000, 1500, 2000, 2500 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=11, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 11 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 11 with n_components=500, coef0=100: KNN: 4.96%, NC: 8.72%
Poly deg 11 with n_components=1000, coef0=100: KNN: 4.84%, NC: 7.84%
Poly deg 11 with n_components=1500, coef0=100: KNN: 5.72%, NC: 7.44%
Poly deg 11 with n_components=2000, coef0=100: KNN: 5.60%, NC: 6.28%
Poly deg 11 with n_components=2500, coef0=100: KNN: 1.96%, NC: 5.84%
Poly deg 11 with n_components=500, coef0=10: KNN: 5.88%, NC: 10.32%
Poly deg 11 with n_components=1000, coef0=10: KNN: 5.96%, NC: 9.08%
Poly deg 11 with n_components=1500, coef0=10: KNN: 6.04%, NC: 8.80%
Poly deg 11 with n_components=2000, coef0=10: KNN: 6.24%, NC: 7.84%
Poly deg 11 with n_components=2500, coef0=10: KNN: 2.28%, NC: 5.72%
Poly deg 11 with n_components=500, coef0=1: KNN: 7.04%, NC: 9.64%
Poly deg 11 with n_components=1000, coef0=1: KNN: 7.68%, NC: 9.20%
Poly deg 11 with n_components=1500, coef0=1: KNN: 7.56%, NC: 8.20%
Poly deg 11 with n_components=2000, coef0=1: KNN: 6.84%, NC: 7.76%
Poly deg 11 with n_components=2500, coef0=1: KNN:

* `n_components=500, coef0=10: KNN: 5.88%, NC: 10.32%`
* `n_components=500, coef0=1: KNN: 7.04%, NC: 9.64%`

In [59]:
for coef0 in [ 0.5, 1, 3, 5, 7, 10, 12 ]:
    for n_components in [ 300, 400, 500, 600, 700 ]:
        kpca_model = KernelPCA(n_components=n_components, kernel='poly', degree=11, coef0=coef0)
        knn_acc, nc_acc = kpca_plus_lda(kpca_model)

        print(f'Poly deg 11 with n_components={n_components}, coef0={coef0}: KNN: {100 * knn_acc:.2f}%, NC: {100 * nc_acc:.2f}%')

Poly deg 11 with n_components=300, coef0=0.5: KNN: 7.32%, NC: 9.24%
Poly deg 11 with n_components=400, coef0=0.5: KNN: 7.64%, NC: 8.88%
Poly deg 11 with n_components=500, coef0=0.5: KNN: 7.72%, NC: 8.40%
Poly deg 11 with n_components=600, coef0=0.5: KNN: 7.72%, NC: 8.24%
Poly deg 11 with n_components=700, coef0=0.5: KNN: 7.56%, NC: 8.16%
Poly deg 11 with n_components=300, coef0=1: KNN: 7.80%, NC: 10.44%
Poly deg 11 with n_components=400, coef0=1: KNN: 7.32%, NC: 9.48%
Poly deg 11 with n_components=500, coef0=1: KNN: 7.04%, NC: 9.64%
Poly deg 11 with n_components=600, coef0=1: KNN: 7.12%, NC: 9.52%
Poly deg 11 with n_components=700, coef0=1: KNN: 7.52%, NC: 9.72%
Poly deg 11 with n_components=300, coef0=3: KNN: 5.32%, NC: 12.12%
Poly deg 11 with n_components=400, coef0=3: KNN: 5.92%, NC: 10.60%
Poly deg 11 with n_components=500, coef0=3: KNN: 6.12%, NC: 10.96%
Poly deg 11 with n_components=600, coef0=3: KNN: 6.48%, NC: 10.80%
Poly deg 11 with n_components=700, coef0=3: KNN: 6.84%, NC: 1

`n_components=300, coef0=3: KNN: 5.32%, NC: 12.12%`

## Σύνοψη
Συλλέγω τις καλύτερες παραμέτρους από όλα τα kernels και τα τρέχω στο test set.

In [64]:
def kpca_plus_lda_test(kpca_model, lda_model_in=None):
    X_train_kpca = kpca_model.fit_transform(X_train)
    X_test_kpca = kpca_model.transform(X_test)
    
    lda_model = lda_model_in if lda_model_in is not None else LinearDiscriminantAnalysis()
    
    X_train_lda = lda_model.fit_transform(X_train_kpca, y_train)
    X_test_lda = lda_model.transform(X_test_kpca)
    
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train_lda, y_train)

    y_pred = knn.predict(X_test_lda)
    knn_acc = accuracy_score(y_pred, y_test)

    nc = NearestCentroid()
    nc.fit(X_train_lda, y_train)

    y_pred = nc.predict(X_test_lda)
    nc_acc = accuracy_score(y_pred, y_test)
    
    return knn_acc, nc_acc

print('Accuracies on the test data.')

kpca_rbf = KernelPCA(n_components=900, kernel='rbf', gamma=0.0012)
knn_acc, nc_acc = kpca_plus_lda_test(kpca_rbf)
print(f'KPCA RBF: KNN: {100 * knn_acc:.2f}%, NCC: {100 * nc_acc:.2f}%')

kpca_poly2 = KernelPCA(n_components=400, kernel='poly', degree=2, coef0=0.007)
knn_acc, nc_acc = kpca_plus_lda_test(kpca_poly2)
print(f'KPCA poly deg 2: KNN: {100 * knn_acc:.2f}%, NCC: {100 * nc_acc:.2f}%')

kpca_poly3 = KernelPCA(n_components=400, kernel='poly', degree=3, coef0=0.09)
knn_acc, nc_acc = kpca_plus_lda_test(kpca_poly3)
print(f'KPCA poly deg 3: KNN: {100 * knn_acc:.2f}%, NCC: {100 * nc_acc:.2f}%')

kpca_poly5 = KernelPCA(n_components=300, kernel='poly', degree=5, coef0=0.9)
knn_acc, nc_acc = kpca_plus_lda_test(kpca_poly5)
print(f'KPCA poly deg 5: KNN: {100 * knn_acc:.2f}%, NCC: {100 * nc_acc:.2f}%')

kpca_poly7 = KernelPCA(n_components=400, kernel='poly', degree=7, coef0=1)
knn_acc, nc_acc = kpca_plus_lda_test(kpca_poly7)
print(f'KPCA poly deg 7: KNN: {100 * knn_acc:.2f}%, NCC: {100 * nc_acc:.2f}%')

kpca_poly9 = KernelPCA(n_components=500, kernel='poly', degree=9, coef0=1)
knn_acc, nc_acc = kpca_plus_lda_test(kpca_poly9)
print(f'KPCA poly deg 9: KNN: {100 * knn_acc:.2f}%, NCC: {100 * nc_acc:.2f}%')

kpca_poly11 = KernelPCA(n_components=300, kernel='poly', degree=11, coef0=3)
knn_acc, nc_acc = kpca_plus_lda_test(kpca_poly11)
print(f'KPCA poly deg 11: KNN: {100 * knn_acc:.2f}%, NCC: {100 * nc_acc:.2f}%')



Accuracies on the test data.
KPCA RBF: KNN: 7.53%, NCC: 10.67%
KPCA poly deg 2: KNN: 5.81%, NCC: 10.72%
KPCA poly deg 3: KNN: 6.66%, NCC: 10.80%
KPCA poly deg 5: KNN: 5.94%, NCC: 11.44%
KPCA poly deg 7: KNN: 6.34%, NCC: 11.13%
KPCA poly deg 9: KNN: 7.22%, NCC: 10.89%
KPCA poly deg 11: KNN: 5.94%, NCC: 11.32%


Πίνακας αποτελεσμάτων για το CIFAR-10 στα test data:

| Kernel      | KNN Acc    | NCC Acc  |
| :---        | ---    | ---    |
| original    | 7.15% | 8.79% |
| PCA         | 7.71% | 8.68% |
| PCA+LDA     | 6.58% | 11.06% |
| Poly deg 2  | 5.81% | 10.72% |
| Poly deg 3  | 6.66% | 10.80% |
| Poly deg 5  | 5.94% | 11.44% |
| Poly deg 7  | 6.34% | 11.13% |
| Poly deg 9  | 7.22% | 10.89% |
| Poly deg 11 | 5.94% | 11.32% |
| RBF         | 7.53% | 10.67% |

## Σύγκριση με SVM
Δοκιμάζω μερικά SVM για σύγκριση:

In [65]:
from sklearn.svm import SVC

def train_test_SVM(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    return accuracy_score(y_pred, y_val)

In [66]:
linear_svc = SVC(kernel='linear')
linear_svc.fit(X_train, y_train) # same results for any C

y_pred = linear_svc.predict(X_test)
acc = accuracy_score(y_pred, y_test)
print(f'Linear SVM: {100 * acc:.2f}%')

Linear SVM: 10.95%


In [71]:
for gamma in ['scale', 'auto']:
    for C in [1e-3, 1e-2, .1, 1, 10, 100, 1e3]:
        rbf_svc = SVC(kernel='rbf', gamma=gamma, C=C)
        
        acc = train_test_SVM(rbf_svc)
        print(f'RBF SVM with gamma={gamma}, C={C}: {100 * acc:.2f}%')        


RBF SVM with gamma=scale, C=0.001: 0.92%
RBF SVM with gamma=scale, C=0.01: 0.92%
RBF SVM with gamma=scale, C=0.1: 0.92%
RBF SVM with gamma=scale, C=1: 11.48%
RBF SVM with gamma=scale, C=10: 13.72%
RBF SVM with gamma=scale, C=100: 13.68%
RBF SVM with gamma=scale, C=1000.0: 13.68%
RBF SVM with gamma=auto, C=0.001: 0.92%
RBF SVM with gamma=auto, C=0.01: 0.92%
RBF SVM with gamma=auto, C=0.1: 0.92%
RBF SVM with gamma=auto, C=1: 3.32%
RBF SVM with gamma=auto, C=10: 12.20%
RBF SVM with gamma=auto, C=100: 12.68%
RBF SVM with gamma=auto, C=1000.0: 12.16%


In [70]:
for gamma in [.1, 1e-2, 1e-3, 1e-4, 1e-5]:
    for C in [1, 10, 100, 1e3, 1e4]:
        rbf_svc = SVC(kernel='rbf', gamma=gamma, C=C)
        
        acc = train_test_SVM(rbf_svc)
        print(f'RBF SVM with gamma={gamma}, C={C}: {100 * acc:.2f}%')    

RBF SVM with gamma=0.1, C=1: 0.92%
RBF SVM with gamma=0.1, C=10: 0.92%
RBF SVM with gamma=0.1, C=100: 0.92%
RBF SVM with gamma=0.1, C=1000.0: 0.92%
RBF SVM with gamma=0.1, C=10000.0: 0.92%
RBF SVM with gamma=0.01, C=1: 11.44%
RBF SVM with gamma=0.01, C=10: 12.68%
RBF SVM with gamma=0.01, C=100: 12.68%
RBF SVM with gamma=0.01, C=1000.0: 12.68%
RBF SVM with gamma=0.01, C=10000.0: 12.68%
RBF SVM with gamma=0.001, C=1: 8.00%
RBF SVM with gamma=0.001, C=10: 13.56%
RBF SVM with gamma=0.001, C=100: 13.20%
RBF SVM with gamma=0.001, C=1000.0: 13.16%
RBF SVM with gamma=0.001, C=10000.0: 13.16%
RBF SVM with gamma=0.0001, C=1: 1.44%
RBF SVM with gamma=0.0001, C=10: 8.24%
RBF SVM with gamma=0.0001, C=100: 12.96%
RBF SVM with gamma=0.0001, C=1000.0: 12.60%
RBF SVM with gamma=0.0001, C=10000.0: 12.52%
RBF SVM with gamma=1e-05, C=1: 0.92%
RBF SVM with gamma=1e-05, C=10: 1.56%
RBF SVM with gamma=1e-05, C=100: 8.44%
RBF SVM with gamma=1e-05, C=1000.0: 12.48%
RBF SVM with gamma=1e-05, C=10000.0: 12.12%


`gamma=0.001, C=10: 13.56%`

In [72]:
for gamma in [.0006, .0008, .001, .0012, .0014]:
    for C in [6, 8, 10, 12, 14]:
        rbf_svc = SVC(kernel='rbf', gamma=gamma, C=C)
        
        acc = train_test_SVM(rbf_svc)
        print(f'RBF SVM with gamma={gamma}, C={C}: {100 * acc:.2f}%')    

RBF SVM with gamma=0.0006, C=6: 12.52%
RBF SVM with gamma=0.0006, C=8: 13.16%
RBF SVM with gamma=0.0006, C=10: 13.08%
RBF SVM with gamma=0.0006, C=12: 13.36%
RBF SVM with gamma=0.0006, C=14: 13.40%
RBF SVM with gamma=0.0008, C=6: 13.04%
RBF SVM with gamma=0.0008, C=8: 13.12%
RBF SVM with gamma=0.0008, C=10: 13.32%
RBF SVM with gamma=0.0008, C=12: 13.52%
RBF SVM with gamma=0.0008, C=14: 13.92%
RBF SVM with gamma=0.001, C=6: 13.32%
RBF SVM with gamma=0.001, C=8: 13.40%
RBF SVM with gamma=0.001, C=10: 13.56%
RBF SVM with gamma=0.001, C=12: 13.68%
RBF SVM with gamma=0.001, C=14: 13.40%
RBF SVM with gamma=0.0012, C=6: 13.52%
RBF SVM with gamma=0.0012, C=8: 13.60%
RBF SVM with gamma=0.0012, C=10: 13.60%
RBF SVM with gamma=0.0012, C=12: 13.20%
RBF SVM with gamma=0.0012, C=14: 13.48%
RBF SVM with gamma=0.0014, C=6: 13.84%
RBF SVM with gamma=0.0014, C=8: 13.88%
RBF SVM with gamma=0.0014, C=10: 13.48%
RBF SVM with gamma=0.0014, C=12: 13.32%
RBF SVM with gamma=0.0014, C=14: 13.48%


`gamma=0.0014, C=8: 13.88%`

In [73]:
rbf_svc = SVC(kernel='rbf', gamma=0.0014, C=8)
rbf_svc.fit(X_train, y_train) # same results for any C

y_pred = rbf_svc.predict(X_test)
acc = accuracy_score(y_pred, y_test)
print(f'RBF SVM with gamma=0.0014, C=8: {100 * acc:.2f}%')

RBF SVM with gamma=0.0014, C=8: 13.44%
